In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os

In [2]:
def distance(vector1, vector2):
    vector1 = np.array(vector1, dtype='f')    
    vector2 = np.array(vector2, dtype='f')
    return np.linalg.norm(vector1-vector2)

In [3]:
houses = pd.read_csv("train.csv", sep=",")
y = houses['SalePrice'].values
X1 = []
w1 = [1]
i = 2
houses_int64 = houses.select_dtypes(include=["int64"])
print(houses_int64.columns)
for column in houses_int64.columns:
    if ((column == 'Id') | (column == 'SalePrice')):
        pass
    else:
        w1.append(i)
        i += 1
        x = houses_int64[column].values
        if (x.std() == 0): 
            pass
        else:
            x_scaled = (x - x.mean()) / x.std()
        X1.append(x_scaled)
X = np.array(X1)
w = np.array(w1)
lmbd = 10
L = 0.001 
N = len(w) - 1
print(N)
epochs = 1000 

w_updated = w
difference = 1.
iteration = 1
tolerance=1e-5

for i in range(epochs): 
    w[0] = w[0] + L * sum(y - w[0] - sum(w[k + 1] * X[k] for k in range(w.shape[0] - 1)) + lmbd * np.sign(w[0]))
    for j in range(w.shape[0]):        
        if (j == 0):
            pass
        else:
            w[j] = w[j] + L * sum(X[j - 1] * (y - w[0] - sum(w[k + 1] * X[k] for k in range(w.shape[0] - 1)) + lmbd * np.sign(w[j])))
"""
while(difference > tolerance):
    w_updated[0] = w[0] + L  * sum(y - w[0] - sum(w[k + 1] * X[k] for k in range(w.shape[0] - 1)) + lmbd * np.sign(w[0]))
    for j in range(w.shape[0]):        
        if (j == 0):
            pass
        else:
            w_updated[j] = w[j] + L * sum(X[j - 1] * (y - w[0] - sum(w[k + 1] * X[k] for k in range(w.shape[0] - 1)) + lmbd * np.sign(w[j])))
    difference = distance(w, w_updated)
    w = w_updated
    if(iteration >= epochs):
        break;
    iteration = iteration + 1
"""    
   
print(w)
result = houses[['Id', 'SalePrice']]
result['predicted_gradient_solution'] = w[0] + sum(w[k + 1] * X[k] for k in range(w.shape[0] - 1))
print(result)

Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')
33
[183345  -5293   6319  21661   6178  11780   2267  13528    113   4718
   6340  13753  21908   1956   8673   2351     40   1695   -831 -10509
  -1813   8028   1764   1679   4914   1666   -870    623   1199   3481
   5320     35  -1254  -1396]
      Id  SalePrice  predicted_gradient_solution
0      0     120000                117429.813805
1      1     232600                233528.202259
2      2     175500                188900.705966
3      3   

/home/ann/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score
from sklearn.model_selection import train_test_split

n = result.shape[0]

MSE = 0
for i in range(n):
    MSE = MSE + (result.at[i, 'SalePrice'] - result.at[i, 'predicted_gradient_solution']) ** 2
MSE = MSE / n
print("MSE = {}".format(MSE))
        
ymean = np.mean(y)
TSS = ((y - ymean) ** 2).sum()
ESS = ((result['predicted_gradient_solution'] - ymean) ** 2).sum()
detCoef = ESS/TSS
print("R^2 = {}".format(detCoef))

MSE = 1256130588.9558287
R^2 = 0.8234287277055151


In [5]:
#import statsmodels
#import statsmodels.formula.api as smf
#model = smf.ols('SalePrice~MSSubClass, LotArea, OverallQual, OverallCond,YearBuilt, YearRemodAdd, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF,TotalBsmtSF, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea,BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr,KitchenAbvGr, TotRmsAbvGrd, Fireplaces, GarageCars,GarageArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch,ScreenPorch, PoolArea, MiscVal, MoSold, YrSold',data=houses)
#fitted = model.fit()
#print (fitted.summary())

In [6]:
houses_test = pd.read_csv("test.csv", sep=",")

houses_test_int64 = houses_test.select_dtypes(include=["int64"])
X1 = []
for column in houses_test_int64.columns:
    if ((column == 'Id')):
        pass
    else:
        x = houses_test_int64[column].values
        if (x.std() == 0): 
            pass
        else:
            x_scaled = (x - x.mean()) / x.std()
        X1.append(x_scaled)
X = np.array(X1)

r =  w[0] + sum(w[k + 1] * X[k] for k in range(w.shape[0] - 1))
res = pd.DataFrame({"SalePrice": r}, columns=["SalePrice"])
res.index = houses_test['Id']
print(res)

res.to_csv("submission.csv")

         SalePrice
Id                
782  311288.588848
783  215141.933535
784  154791.494427
785  235688.312250
786  284972.260940
787  289282.075659
788  253955.767200
789  167511.169679
790  167256.458881
791  100737.666985
792  109410.433549
793  213536.113622
794  255968.099438
795  368621.852470
796   65182.396661
797  209587.691168
798  105270.251438
799  134395.497492
800  132632.251918
801  155727.750464
802  234169.310926
803   89583.756100
804  128194.179864
805  215408.661847
806   85173.878036
807  311675.255719
808  140192.318633
809  317014.387734
810  207926.674707
811  213109.269913
..             ...
948  108524.899730
949  143215.161485
950  180393.858189
951  195080.187025
952   67545.378462
953  317129.507243
954  242216.241231
955  135508.678750
956  256784.183851
957  283383.435685
958  404026.822025
959  203322.091642
960  211605.282509
961  200906.358736
962  314878.032660
963    5766.059915
964  302841.309686
965  134517.311239
966  252587.580369
967  108465.